# 3rd Level Model Structure: Single Stage Reactive Distillation

In [1]:
import sys
import os
import pickle
sys.path.append(os.path.abspath('..'))

import numpy as np
from matplotlib import pyplot as plt
import ipywidgets as widgets
from copy import deepcopy

In [2]:
from pyomo import environ as pe
from global_sets.component import m

from stages.reactive_stage import reactive_stage_rule
from stages.condenser_stage import condenser_stage_rule
from stages.non_reactive_stage import non_reactive_stage_rule

from utility.display_utility import trans_product_mole, trans_product_mass, beautify2
from utility.model_utility import add_dual, update_dual, check_DOF
from utility.data_utility import cal_cnumber

model = pe.ConcreteModel()

# Global Set

In [3]:
model.TRAY = pe.RangeSet(1,1)

# Construct Reactive Stages

In [4]:
model.reactive = pe.Block(model.TRAY,rule=reactive_stage_rule)

> Importing Reactive Stage......
> Adding the following local variable:
------------------------------------
| reactive[1].T_F
| reactive[1].P
| reactive[1].cat
| reactive[1].Q_main
| reactive[1].x_
| reactive[1].y_
| reactive[1].x
| reactive[1].y
| reactive[1].z
| reactive[1].L
| reactive[1].V
| reactive[1].F
| reactive[1].H_L_
| reactive[1].H_V_
| reactive[1].H_L
| reactive[1].H_V
| reactive[1].T
| reactive[1].H_F
| reactive[1].f_V
| reactive[1].f_L
| reactive[1].r_total_comp
------------------------------------

> Importing Kinetics Blocks......
> Adding the following local variable:
--------------------------------------------------
| reactive[1].kinetics_block.k_FT
| reactive[1].kinetics_block.r_FT_total
| reactive[1].kinetics_block.g0_FT
| reactive[1].kinetics_block.alpha
| reactive[1].kinetics_block.r_FT_cnum
| reactive[1].kinetics_block.r_FT_comp
| reactive[1].kinetics_block.k_WGS
| reactive[1].kinetics_block.Ke_WGS
| reactive[1].kinetics_block.r_WGS
| reactive[1].kinetics_bloc

# Construct a single condenser

In [5]:
model.condenser = pe.Block(rule=condenser_stage_rule)

| Importing Condenser Stage......
| Adding the following local variable:
------------------------------------
| condenser.T
| condenser.T_F
| condenser.P
| condenser.Q_main
| condenser.x_
| condenser.y_
| condenser.x
| condenser.y
| condenser.z
| condenser.L
| condenser.W
| condenser.V
| condenser.F
| condenser.H_L_
| condenser.H_V_
| condenser.H_L
| condenser.H_V
| condenser.H_F
| condenser.f_V
| condenser.f_L
------------------------------------

> Importing Energy Blocks......
> Adding the following local variable:
--------------------------------------------------
| condenser.energy_block.dH_F
| condenser.energy_block.dH_V
| condenser.energy_block.dH_L
| condenser.energy_block.dH_vap
--------------------------------------------------

> Importing VLE Blocks......
> Adding the following local variable:
--------------------------------------------------
| condenser.VLE_block.n_ave
| condenser.VLE_block.n_ave_cal
| condenser.VLE_block.Hen
| condenser.VLE_block.Hen0
| condenser.VLE_blo

# Construct a reboiler

In [6]:
model.reboiler = pe.Block(rule=non_reactive_stage_rule)

> Importing Non Reactive Stage......
> Adding the following local variable:
------------------------------------
| reboiler.T_F
| reboiler.P
| reboiler.Q_main
| reboiler.x_
| reboiler.y_
| reboiler.x
| reboiler.y
| reboiler.z
| reboiler.L
| reboiler.V
| reboiler.F
| reboiler.H_L_
| reboiler.H_V_
| reboiler.H_L
| reboiler.H_V
| reboiler.T
| reboiler.H_F
| reboiler.f_V
| reboiler.f_L
------------------------------------

> Importing Energy Blocks......
> Adding the following local variable:
--------------------------------------------------
| reboiler.energy_block.dH_F
| reboiler.energy_block.dH_V
| reboiler.energy_block.dH_L
| reboiler.energy_block.dH_vap
--------------------------------------------------

> Importing VLE Blocks......
> Adding the following local variable:
--------------------------------------------------
| reboiler.VLE_block.P_VLE
| reboiler.VLE_block.n_ave
| reboiler.VLE_block.n_ave_cal
| reboiler.VLE_block.Hen
| reboiler.VLE_block.Hen0
| reboiler.VLE_block.gamma
| r

# Linking Stage Variables

### Vapor Between Reactive Stages

In [7]:
def V_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j].V['in'] == model.reactive[j+1].V['out']
model.V_between_con = pe.Constraint(model.TRAY,rule=V_between_rule)

def Vy_between_rule(model,j,i):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j].y_['in',i] == model.reactive[j+1].y[i]
model.Vy_between_con = pe.Constraint(model.TRAY,m.COMP_TOTAL,rule=Vy_between_rule)

def Vh_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j].H_V_['in'] == model.reactive[j+1].H_V
model.Vh_between_con = pe.Constraint(model.TRAY,rule=Vh_between_rule)

### Liquid Between Reactive Stages

In [8]:
def L_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j+1].L['in'] == model.reactive[j].L['out']
model.L_between_con = pe.Constraint(model.TRAY,rule=L_between_rule)

def Lx_between_rule(model,j,i):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j+1].x_['in',i] == model.reactive[j].x[i]
model.Ly_between_con = pe.Constraint(model.TRAY,m.COMP_TOTAL,rule=Lx_between_rule)

def Lh_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j+1].H_L_['in'] == model.reactive[j].H_L
model.Lh_between_con = pe.Constraint(model.TRAY,rule=Lh_between_rule)

### Condenser

In [9]:
def V_condenser_rule(model):
    return model.reactive[model.TRAY.first()].V['out'] == model.condenser.V['in']
model.V_condenser_con = pe.Constraint(rule=V_condenser_rule)

def Vy_condenser_rule(model,i):
    return model.reactive[model.TRAY.first()].y[i] == model.condenser.y_['in',i]
model.Vy_condenser_con = pe.Constraint(m.COMP_TOTAL,rule=Vy_condenser_rule)

def Vh_condenser_rule(model):
    return model.reactive[model.TRAY.first()].H_V == model.condenser.H_V_['in']
model.Vh_condenser_con = pe.Constraint(rule=Vh_condenser_rule)

In [10]:
def L_condenser_rule(model):
    return model.reactive[model.TRAY.first()].L['in'] == model.condenser.L['out']
model.L_condenser_con = pe.Constraint(rule=L_condenser_rule)

def Lx_condenser_rule(model,i):
    return model.reactive[model.TRAY.first()].x_['in',i] == model.condenser.x[i]
model.Lx_condenser_con = pe.Constraint(m.COMP_TOTAL,rule=Lx_condenser_rule)

def Lh_condenser_rule(model):
    return model.reactive[model.TRAY.first()].H_L_['in'] == model.condenser.H_L
model.Lh_condenser_con = pe.Constraint(rule=Lh_condenser_rule)

### Reboiler

In [11]:
def V_reboiler_rule(model):
    return model.reactive[model.TRAY.last()].V['in'] == model.reboiler.V['out']
model.V_reboiler_con = pe.Constraint(rule=V_reboiler_rule)

def Vy_reboiler_rule(model,i):
    return model.reactive[model.TRAY.last()].y_['in',i] == model.reboiler.y[i]
model.Vy_reboiler_con = pe.Constraint(m.COMP_TOTAL,rule=Vy_reboiler_rule)

def Vh_reboiler_rule(model):
    return model.reactive[model.TRAY.last()].H_V_['in'] == model.reboiler.H_V
model.Vh_reboiler_con = pe.Constraint(rule=Vh_reboiler_rule)

In [12]:
def L_reboiler_rule(model):
    return model.reactive[model.TRAY.last()].L['out'] == model.reboiler.L['in']
model.L_reboiler_con = pe.Constraint(rule=L_reboiler_rule)

def Lx_reboiler_rule(model,i):
    return model.reactive[model.TRAY.last()].x[i] == model.reboiler.x_['in',i]
model.Lx_reboiler_con = pe.Constraint(m.COMP_TOTAL,rule=Lx_reboiler_rule)

def Lh_reboiler_rule(model):
    return model.reactive[model.TRAY.last()].H_L == model.reboiler.H_L_['in']
model.Lh_reboiler_con = pe.Constraint(rule=Lh_reboiler_rule)

In [13]:
# model.obj = pe.Objective(expr = sum(model.reactive[j].T - model.reactive[j].MPCC.pf for j in model.TRAY) ,sense=pe.maximize)
model.obj = pe.Objective(expr = sum(model.reactive[j].T for j in model.TRAY) ,sense=pe.maximize)

In [14]:
add_dual(pe,model)

Created the follow pyomo suffixes:
ipopt_zL_out, ipopt_zU_out, ipopt_zL_in, ipopt_zU_in, dual


# Load from single stage solutions

In [15]:
# with open('../saved_solutions/reactive_flash_MPCC_P_pf.pickle', 'rb') as f:
#     results_imported = pickle.load(f)
# results_changed = deepcopy(results_imported)

### Duplicate variable solution and bounds multiplier

In [16]:
# results_changed.Solution.Variable = {}
# for i in list(results_imported.Solution.Variable.keys()):
#     for j in model.TRAY:
#         results_changed.Solution.Variable['reactive[{}].{}'.format(j,i)] = \
#         results_imported.Solution.Variable[i]

### Duplicate constraint multiplier

In [17]:
# results_changed.Solution.Constraint = {}
# for i in list(results_imported.Solution.Constraint.keys()):
#     for j in model.TRAY:
#         results_changed.Solution.Constraint['reactive[{}].{}'.format(j,i)] = \
#         results_imported.Solution.Constraint[i]

### Load changed solution into current model

In [18]:
# model.solutions.load_from(results_changed)

# Fixing Redundent Stream Variables

In [19]:
# condenser
model.condenser.VLE_block.n_ave.fix(4)

model.condenser.F.fix(0)
model.condenser.T_F.fix(300+273.15)
model.condenser.z.fix(0)

model.condenser.V['P'].fix(0)
model.condenser.L['in'].fix(0)
for i in m.COMP_TOTAL: model.condenser.x_['in',i].fix(0)
model.condenser.H_L_['in'].fix(0)

In [20]:
# 'reboiler' fixing last stage V_in

# model.reactive[model.TRAY.last()].V['in'].fix(0)
# for i in m.COMP_TOTAL: model.reactive[model.TRAY.last()].y_['in',i].fix(0)
# model.reactive[model.TRAY.last()].H_V_['in'].fix(0)

model.reboiler.VLE_block.n_ave.fix(20)

model.reboiler.F.fix(0)
model.reboiler.T_F.fix(300+273.15)
model.reboiler.z.fix(0)

model.reboiler.V['in'].fix(0)
for i in m.COMP_TOTAL: model.reboiler.y_['in',i].fix(0)
model.reboiler.H_V_['in'].fix(0)

# Load Operating Parameters

In [21]:
# condenser
model.condenser.P.fix(19)
model.condenser.T.fix(30+273.15)
model.condenser.L['out'].fix(0)

# reactive stage
for j in model.reactive:
    model.reactive[j].cat.fix(5000)
    model.reactive[j].P.fix(20)
    model.reactive[j].VLE_block.n_ave.fix(20)
    
    model.reactive[j].F.fix(1)
    model.reactive[j].T_F.fix(200+273.15)
    model.reactive[j].z['CO'].fix(1/(1+2))
    model.reactive[j].z['H2'].fix(2/(1+2))
    model.reactive[j].z['C30H62'].fix(0)
    model.reactive[j].V['P'].fix(0)
    model.reactive[j].L['P'].fix(0)
    # model.reactive[j].Q_main.fix(0)
    model.reactive[j].T.setub(300+273.15)
    
# reboiler
model.reboiler.L['P'].fix(0)
model.reboiler.V['P'].fix(0)
model.reboiler.P.fix(20)
model.reboiler.T.setub(350+273.15)

In [22]:
check_DOF(pe,model)

Active Equality Constraints:	 3424
Active Inequality Constraints:	 0
Active Variables:		 3617
Fixed Variables:		 191
DOF:				 2


In [34]:
opt = pe.SolverFactory('ipopt')

opt.options['print_user_options'] = 'yes'
opt.options['linear_solver'] = 'ma86'

opt.options['linear_system_scaling '] = 'mc19'
opt.options['linear_scaling_on_demand '] = 'no'
# opt.options['recalc_y'] = 'yes'
# opt.options['bound_relax_factor'] = 0
# opt.options['halt_on_ampl_error'] = 'yes'

opt.options['max_iter'] = 7000
results = opt.solve(model,tee=True)
update_dual(pe,model)

Ipopt 3.12.8: print_user_options=yes
linear_solver=ma86
linear_system_scaling =mc19
linear_scaling_on_demand =no
max_iter=7000


List of user-set options:

                                    Name   Value                used
                linear_scaling_on_demand = no                    yes
                           linear_solver = ma86                  yes
                   linear_system_scaling = mc19                  yes
                                max_iter = 7000                  yes
                      print_user_options = yes                   yes

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.8, running with

  73 -5.7299015e+02 2.09e+01 3.59e+16  -1.0 2.28e+05    -  7.53e-01 1.05e-04h  6
  74 -5.7299015e+02 2.09e+01 3.38e+16  -1.0 1.95e+05    -  1.97e-02 1.03e-04h  7
  75 -5.7299015e+02 2.09e+01 1.05e+16  -1.0 1.97e+05    -  3.11e-01 1.04e-04h  7
  76 -5.7299015e+02 2.09e+01 1.05e+16  -1.0 1.98e+05    -  4.04e-03 5.21e-05h  8
  77 -5.7299016e+02 2.09e+01 1.05e+16  -1.0 1.99e+05    -  1.22e-04 2.09e-04f  6
  78 -5.7299017e+02 2.09e+01 1.05e+16  -1.0 2.02e+05    -  2.22e-04 2.12e-04f  6
  79 -5.7299019e+02 2.09e+01 1.05e+16  -1.0 2.06e+05    -  4.29e-04 2.16e-04f  6
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  80 -5.7299020e+02 2.09e+01 1.05e+16  -1.0 2.10e+05    -  6.76e-04 2.23e-04h  6
  81 -5.7299023e+02 2.09e+01 1.05e+16  -1.0 2.13e+05    -  1.89e-02 2.28e-04h  6
  82 -5.7299027e+02 2.09e+01 1.05e+16  -1.0 2.16e+05    -  1.14e-03 4.66e-04h  5
  83 -5.7299123e+02 3.30e+01 1.05e+16  -1.0 2.22e+05    -  6.34e-02 7.84e-03w  1
  84 -5.7299389e+02 1.05e+02

 168 -5.6976479e+02 7.25e-01 3.14e+08  -1.0 2.15e+03    -  1.00e+00 3.23e-03h  8
 169 -5.6975596e+02 7.23e-01 3.42e+08  -1.0 4.28e+03    -  4.53e-01 3.23e-03h  9
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 170 -5.6974703e+02 7.20e-01 4.10e+08  -1.0 4.53e+03    -  1.00e+00 3.23e-03h  9
 171 -5.6973815e+02 7.18e-01 4.45e+08  -1.0 4.32e+03    -  4.48e-01 3.23e-03h  9
 172 -5.6972920e+02 7.16e-01 5.33e+08  -1.0 4.61e+03    -  1.00e+00 3.23e-03h  9
 173 -5.6972028e+02 7.13e-01 5.78e+08  -1.0 4.36e+03    -  4.43e-01 3.24e-03h  9
 174 -5.6971133e+02 7.11e-01 6.92e+08  -1.0 4.69e+03    -  1.00e+00 3.24e-03h  9
 175 -5.6970241e+02 7.09e-01 7.50e+08  -1.0 4.40e+03    -  4.39e-01 3.24e-03h  9
 176 -5.6969348e+02 7.07e-01 8.96e+08  -1.0 4.76e+03    -  1.00e+00 3.24e-03h  9
 177 -5.6968458e+02 7.04e-01 9.70e+08  -1.0 4.44e+03    -  4.37e-01 3.24e-03h  9
 178 -5.6741785e+02 1.72e+02 9.55e+09  -1.0 4.83e+03    -  1.00e+00 8.25e-01w  1
 179 -5.6726632e+02 1.36e+02

 262 -5.6919407e+02 5.79e-01 1.33e+12  -1.0 3.37e+03    -  3.64e-01 6.39e-03h  8
 263 -5.6918022e+02 5.75e-01 1.53e+12  -1.0 3.34e+03    -  1.00e+00 6.32e-03h  8
 264 -5.6916580e+02 5.72e-01 1.60e+12  -1.0 3.15e+03    -  3.55e-01 6.57e-03h  8
 265 -5.6915170e+02 5.68e-01 1.84e+12  -1.0 3.11e+03    -  1.00e+00 6.51e-03h  8
 266 -5.6913722e+02 5.64e-01 1.93e+12  -1.0 2.98e+03    -  3.46e-01 6.68e-03h  8
 267 -5.6912303e+02 5.61e-01 2.22e+12  -1.0 2.94e+03    -  1.00e+00 6.63e-03h  8
 268 -5.6910862e+02 5.57e-01 2.32e+12  -1.0 2.83e+03    -  3.36e-01 6.73e-03h  8
 269 -5.6729392e+02 5.98e+01 9.68e+12  -1.0 2.79e+03    -  1.00e+00 8.59e-01w  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 270 -5.6722145e+02 4.93e+01 6.27e+12  -1.0 2.91e+03    -  1.78e-01 3.37e-01w  1
 271 -5.6712497e+02 3.76e+01 4.16e+12  -1.0 1.87e+03    -  1.79e-01 3.07e-01w  1
 272 -5.6909445e+02 5.53e-01 2.67e+12  -1.0 5.42e+04  -1.5 1.00e+00 6.71e-03h  7
 273 -5.6908020e+02 5.50e-01

 362 -5.6952313e+02 1.48e+00 1.16e+04  -8.6 6.80e+06    -  2.80e-05 2.73e-06h  1
 363 -5.6952313e+02 1.46e+00 1.16e+04  -8.6 5.06e+06    -  6.39e-06 4.78e-06h  7
 364r-5.6952313e+02 1.46e+00 1.00e+03   0.2 0.00e+00    -  0.00e+00 3.92e-07R 11
 365r-5.6952313e+02 1.46e+00 1.19e+03   0.2 3.39e+04    -  3.00e-04 8.40e-08f  1
 366r-5.6954367e+02 8.67e-01 7.07e+03   0.2 2.70e+04    -  7.62e-03 6.02e-04f  1
 367 -5.6954397e+02 8.97e-01 3.04e+01  -8.6 1.76e+05    -  7.49e-03 1.30e-04h  1
 368r-5.6954397e+02 8.97e-01 9.99e+02  -0.0 0.00e+00    -  0.00e+00 7.49e-10R 19
 369r-5.6954397e+02 8.97e-01 9.99e+02  -0.0 1.07e+04    -  1.69e-04 4.24e-09f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 370r-5.6956585e+02 5.63e-01 3.65e+03  -0.0 8.89e+03    -  4.87e-03 5.02e-04f  1
 371 -5.6957627e+02 3.85e+01 1.08e+02  -8.6 3.90e+05    -  2.41e-04 5.02e-03h  1
 372 -5.6958750e+02 3.83e+01 3.67e+02  -8.6 6.31e+03    -  7.87e-03 6.54e-03h  1
 373 -5.6957233e+02 3.81e+01

...

2589 -5.5983020e+02 3.29e+01 2.18e+10  -8.6 2.66e+05    -  4.95e-03 3.69e-07w  1
In iteration 2589, 1 Slack too small, adjusting variable bound
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2590 -5.5982988e+02 3.29e+01 4.28e+12  -8.6 2.68e+05    -  6.92e-03 1.85e-05w  1
2591 -5.6810197e+02 6.81e-02 4.62e+08  -8.6 2.68e+05    -  4.85e-03 4.68e-05h 12
2592 -5.6809995e+02 6.81e-02 4.65e+08  -8.6 8.70e+03    -  1.82e-02 4.67e-05h 13
2593 -5.6809793e+02 6.81e-02 4.66e+08  -8.6 8.70e+03    -  6.45e-03 4.67e-05h 13
2594 -5.6809591e+02 6.81e-02 4.90e+08  -8.6 8.70e+03    -  1.36e-01 4.67e-05h 13
2595 -5.6809390e+02 6.81e-02 4.91e+08  -8.6 8.70e+03    -  5.05e-03 4.67e-05h 13
2596 -5.6809188e+02 6.81e-02 4.94e+08  -8.6 8.70e+03    -  1.87e-02 4.67e-05h 13
2597 -5.6808986e+02 6.81e-02 4.95e+08  -8.6 8.70e+03    -  6.73e-03 4.67e-05h 13
2598 -5.6808784e+02 6.81e-02 5.13e+08  -8.6 8.70e+03    -  1.10e-01 4.67e-05h 13
2599 -5.6808582e+02 6.81e-02 5.14e+08  -8.6 8.

2676 -5.6796732e+02 6.79e-02 7.59e+08  -8.6 8.64e+03    -  2.27e-02 4.66e-05h 13
2677 -5.6796532e+02 6.79e-02 7.60e+08  -8.6 8.63e+03    -  1.20e-02 4.66e-05h 13
2678 -5.6796332e+02 6.79e-02 7.90e+08  -8.6 8.63e+03    -  3.40e-01 4.66e-05h 13
2679 -5.5977496e+02 3.22e+01 7.45e+08  -8.6 8.63e+03    -  8.14e-03 1.91e-01w  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2680 -5.5977495e+02 3.22e+01 3.39e+10  -8.6 2.62e+05    -  5.05e-03 2.43e-07w  1
In iteration 2680, 1 Slack too small, adjusting variable bound
2681 -5.5977442e+02 3.22e+01 3.90e+12  -8.6 2.65e+05    -  6.65e-03 3.03e-05w  1
2682 -5.6796132e+02 6.79e-02 7.91e+08  -8.6 2.65e+05    -  8.14e-03 4.66e-05h 12
2683 -5.6795932e+02 6.79e-02 7.93e+08  -8.6 8.63e+03    -  2.32e-02 4.66e-05h 13
2684 -5.6795732e+02 6.79e-02 7.95e+08  -8.6 8.63e+03    -  1.23e-02 4.66e-05h 13
2685 -5.6795532e+02 6.79e-02 8.36e+08  -8.6 8.63e+03    -  3.51e-01 4.66e-05h 13
2686 -5.6795332e+02 6.79e-02 8.37e+08  -8.6 8.

2764 -5.6783399e+02 6.77e-02 1.60e+09  -8.6 8.57e+03    -  1.17e-02 4.65e-05h 13
2765 -5.6783201e+02 6.77e-02 1.61e+09  -8.6 8.56e+03    -  4.75e-02 4.65e-05h 13
2766 -5.6783003e+02 6.77e-02 1.61e+09  -8.6 8.56e+03    -  1.44e-02 4.65e-05h 13
2767 -5.6782805e+02 6.77e-02 1.62e+09  -8.6 8.56e+03    -  1.58e-01 4.65e-05h 13
2768 -5.6782607e+02 6.77e-02 1.62e+09  -8.6 8.56e+03    -  1.20e-02 4.65e-05h 13
2769 -5.6782409e+02 6.77e-02 1.62e+09  -8.6 8.56e+03    -  5.00e-02 4.65e-05h 13
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2770 -5.5971986e+02 3.15e+01 1.54e+09  -8.6 8.56e+03    -  1.45e-02 1.91e-01w  1
2771 -5.5971986e+02 3.15e+01 4.14e+10  -8.6 2.58e+05    -  5.14e-03 2.08e-07w  1
In iteration 2771, 1 Slack too small, adjusting variable bound
2772 -5.5971920e+02 3.15e+01 4.13e+12  -8.6 2.61e+05    -  7.38e-03 3.82e-05w  1
2773 -5.6782211e+02 6.77e-02 1.62e+09  -8.6 2.61e+05    -  1.45e-02 4.65e-05h 12
2774 -5.6782013e+02 6.77e-02 1.63e+09  -8.6 8.

2852 -5.6763936e+02 6.75e-02 1.66e+09  -8.6 8.46e+03    -  6.18e-02 9.28e-05h 12
2853 -5.6763546e+02 6.75e-02 1.66e+09  -8.6 8.46e+03    -  2.09e-02 9.28e-05h 12
2854 -5.6763155e+02 6.75e-02 1.65e+09  -8.6 8.46e+03    -  2.66e-01 9.27e-05h 12
2855 -5.6762765e+02 6.75e-02 1.65e+09  -8.6 8.45e+03    -  1.68e-02 9.27e-05h 12
2856 -5.6762375e+02 6.75e-02 1.65e+09  -8.6 8.45e+03    -  6.69e-02 9.27e-05h 12
2857 -5.6761986e+02 6.74e-02 1.65e+09  -8.6 8.45e+03    -  2.10e-02 9.27e-05h 12
2858 -5.6761596e+02 6.74e-02 1.65e+09  -8.6 8.45e+03    -  2.41e-01 9.27e-05h 12
2859 -5.6761206e+02 6.74e-02 1.65e+09  -8.6 8.45e+03    -  1.73e-02 9.27e-05h 12
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2860 -5.6760817e+02 6.74e-02 1.65e+09  -8.6 8.44e+03    -  7.01e-02 9.27e-05h 12
2861 -5.5963371e+02 3.05e+01 1.57e+09  -8.6 8.44e+03    -  2.13e-02 1.90e-01w  1
2862 -5.5963370e+02 3.05e+01 4.09e+10  -8.6 2.53e+05    -  5.27e-03 2.16e-07w  1
In iteration 2862, 1 Slack t

2945r-5.6733203e+02 5.33e-04 1.00e+03  -3.5 0.00e+00    -  0.00e+00 8.67e-09R  2
2946r-5.6733202e+02 4.76e-04 2.49e+03  -3.5 4.70e+00    -  6.26e-02 9.91e-04f  1
2947 -5.6733202e+02 4.76e-04 3.24e+06  -8.6 1.95e+01    -  5.00e-01 7.72e-08h  2
2948 -5.6733203e+02 4.76e-04 2.92e+11  -8.6 5.07e+05    -  7.71e-03 8.41e-08h  1
2949r-5.6733203e+02 4.76e-04 1.00e+03  -4.1 0.00e+00    -  0.00e+00 7.51e-09R  2
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2950r-5.6733205e+02 4.68e-04 1.25e+03  -4.1 4.63e+01    -  3.56e-02 9.90e-04f  1
2951r-5.6733205e+02 4.68e-04 9.99e+02  -4.5 0.00e+00    -  0.00e+00 1.97e-08R  2
2952r-5.6733208e+02 4.40e-04 9.97e+02  -4.5 1.52e+01    -  7.56e-03 1.85e-03f  1
2953r-5.6733218e+02 4.16e-04 9.88e+02  -4.5 9.31e+00    -  2.64e-03 9.70e-03f  1
2954r-5.6733221e+02 3.86e-04 9.82e+02  -4.5 7.79e+00    -  2.96e-02 4.95e-03f  1
2955 -5.6733187e+02 3.85e-04 1.14e+01  -8.6 3.93e+01    -  3.25e-03 1.75e-03h  1
2956r-5.6733187e+02 3.85e-04

In [24]:
model.reboiler.T.value

617.5018278045042

In [33]:
model.reboiler.V['out'].value

1.307817463879926e-07

In [ ]:
model.re

In [29]:
beautify2(pe,model)

Here comes the result:
----------------------------------------------------------------------------------------------------
		T		 Q			 V			 L
               30.00 	       -17.3943688624 	         0.1850891985 	         0.0000000000
              300.00 	       -47.0548777531 	         0.4568201229 	         0.0000002138
----------------------------------------------------------------------------------------------------
Top
V	 0.18508919852206768
L	 0.01059839683100223
W	 0.2611325275797905
----------------------------------------------------------------------------------------------------
Bottom
L	 2.1383131390407959e-07
----------------------------------------------------------------------------------------------------
Condenser:	Vapor		Liquid		Last Stage	Vapor		Liquid
H2 		0.526%		0.007%		 H2       	0.213%		0.070%
CO 		0.008%		0.000%		 CO       	0.003%		0.001%
CO2 		19.175%		3.726%		 CO2      	7.856%		5.354%
H2O 		0.223%		0.000%		 H2O      	57.254%		59.714%
C2H4 		13.612%		4.251%		 C

In [27]:
model.reactive[1].kinetics_block.r_FT_total.value

0.3044500326847305

In [ ]:
# model.solutions.store_to(results)
# with open('../saved_solutions/3_stage_condenser_240C.pickle','wb') as f:
#     pickle.dump(results,f)

# So, what exactly does adding a reflux do?

In [ ]:
opt.options['warm_start_init_point'] = 'yes'
opt.options['warm_start_bound_push'] = 1e-20
opt.options['warm_start_mult_bound_push'] = 1e-20
opt.options['mu_init'] = 1e-6

In [ ]:
results_anchor = results
update_dual(pe,model)
model.solutions.store_to(results_anchor)

In [ ]:
Refluxrange = np.linspace(0,2,21)
Trange = np.linspace(300+273.15,200+273.15,21)

In [ ]:
cd_data_master = {}
rf_data_master = {}

for t in Trange:
    model.del_component(model.obj)
#     model.obj = pe.Objective(expr = sum(model.reactive[j].T - model.reactive[j].MPCC.pf for j in model.TRAY) ,sense=pe.maximize)
    model.obj = pe.Objective(expr = sum(model.reactive[j].T for j in model.TRAY) ,sense=pe.maximize)
    model.solutions.load_from(results_anchor)
    update_dual(pe,model)
    
    for j in model.reactive:
        model.reactive[j].T.fixed = False
        model.reactive[j].T.setub(t)
        model.reactive[j].T = t
    model.condenser.L['out'].fix(0)
    
    results_anchor = opt.solve(model,tee=False)
    model.solutions.store_to(results_anchor)
    
    model.del_component(model.obj)
#     model.obj = pe.Objective(expr = model.condenser.L['out'] - sum(model.reactive[j].MPCC.pf for j in model.TRAY),sense=pe.maximize)
    model.obj = pe.Objective(expr = model.condenser.L['out'],sense=pe.maximize)
    model.condenser.L['out'].fixed = False
    for j in model.reactive:
        model.reactive[j].T.fixed=True
    
    print('\nWorking on T = {} K'.format(model.reactive[1].T.value))
    print('-'*108)
    
    cd_data = {};
    cd_data['Re'] = []; cd_data['D'] = []; cd_data['V'] = []
    cd_data['x'] = {}; cd_data['y'] = {}; cd_data['g'] = {}; cd_data['d'] = {}; cd_data['b'] = {}
    for i in m.COMP_TOTAL:
        cd_data['x'][i] = []
        cd_data['y'][i] = []
        cd_data['g'][i] = []
        cd_data['d'][i] = []
        cd_data['b'][i] = []

    rf_data = {}
    for j in model.reactive:
        rf_data[j] = {}
        rf_data[j]['r'] = {}; rf_data[j]['b'] = {}; rf_data[j]['x'] = {};rf_data[j]['y'] = {};
        rf_data[j]['T'] = []; rf_data[j]['Q'] = []; rf_data[j]['V'] = []; rf_data[j]['L'] = []; 
        rf_data[j]['r_WGS'] = []; rf_data[j]['r_FT'] = []; rf_data[j]['pf'] = []
        for i in m.COMP_TOTAL:
            rf_data[j]['r'][i] = []
            rf_data[j]['b'][i] = []
            rf_data[j]['x'][i] = []
            rf_data[j]['y'][i] = []       

    for re in Refluxrange:
        model.condenser.L['out'].setub(re)
        results = opt.solve(model,tee=False)
        update_dual(pe,model)
        print('Solved\t|Reflux = {:.3f} kmol/s\t|Vapor = {:.3f} kmol/s\t|Distillate = {:.3f} kmol/s\t|Bottom = {:.3f} kmol/s'.\
              format(model.condenser.L['out'].value,model.condenser.V['out'].value,model.condenser.L['P'].value,model.reboiler.L['out'].value))

        cd_data['V'].append(model.condenser.V['out'].value)
        cd_data['D'].append(model.condenser.L['P'].value)
        cd_data['Re'].append(model.condenser.L['out'].value)

        for i in model.reactive[1].r_total_comp:
            cd_data['x'][i].append(model.condenser.x[i].value)
            cd_data['y'][i].append(model.condenser.y[i].value)
            cd_data['g'][i].append(model.condenser.y[i].value*model.condenser.V['out'].value)
            cd_data['d'][i].append(model.condenser.x[i].value*model.condenser.L['P'].value)
            cd_data['b'][i].append(model.condenser.x[i].value*model.condenser.L['out'].value)

        for j in model.reactive:      
            rf_data[j]['T'].append(model.reactive[j].T.value)
            rf_data[j]['Q'].append(model.reactive[j].Q_main.value)
            rf_data[j]['V'].append(model.reactive[j].V['out'].value)
            rf_data[j]['L'].append(model.reactive[j].L['out'].value)
            rf_data[j]['r_WGS'].append(model.reactive[j].kinetics_block.r_WGS.value)
            rf_data[j]['r_FT'].append(model.reactive[j].kinetics_block.r_FT_total.value)
#             rf_data[j]['pf'].append(model.reactive[j].MPCC.pf.value)

            for i in model.reactive[1].r_total_comp:
                rf_data[j]['r'][i].append(model.reactive[j].r_total_comp[i].value)
                rf_data[j]['b'][i].append(model.reactive[j].x[i].value*model.reactive[j].L['out'].value)
                rf_data[j]['x'][i].append(model.reactive[j].x[i].value)
                rf_data[j]['y'][i].append(model.reactive[j].y[i].value)
        
    cd_data_master[t] = cd_data
    rf_data_master[t] = rf_data

In [ ]:
cnumber_range = range(1,57)

In [ ]:
def trans_cnumber(dic):
    molefraction = {}
    for i in range(1,57):
        molefraction[i] = []
    for i in m.COMP_ORG:
        molefraction[cal_cnumber(i)].append(np.array(dic[i]))
    for i in range(1,57):
        molefraction[i] = np.sum(molefraction[i],0)
    length = len(molefraction[1])
    tmp = {}
    for j in range(length):
        tmp[j] = []
        for i in range(1,57):
            tmp[j].append(molefraction[i][j])
    return tmp

In [ ]:
g_data = {}
d_data = {}
lr_data = {}
l1_data = {}
l2_data = {}
l3_data = {}

cd_x_data = {}
rf_x_data = {}

for t in Trange:

    g_data[t] = trans_cnumber(cd_data_master[t]['g'])
    d_data[t] = trans_cnumber(cd_data_master[t]['d'])
    lr_data[t] = trans_cnumber(cd_data_master[t]['d'])
    l1_data[t] = trans_cnumber(rf_data_master[t][1]['b'])
    l2_data[t] = trans_cnumber(rf_data_master[t][2]['b'])
    l3_data[t] = trans_cnumber(rf_data_master[t][3]['b'])
    
    cd_x_data[t] = trans_cnumber(cd_data_master[t]['x'])
    rf_x_data[t] = {}
    for j in model.reactive:
        rf_x_data[t][j] = trans_cnumber(rf_data_master[t][j]['x'])

In [ ]:
 def plot_distribution(index,temperature):
    fig, (ax,ax2) = plt.subplots(2,1,figsize=(16,12))
    ax.plot(cnumber_range,g_data[temperature][index],'co-')
    ax.plot(cnumber_range,d_data[temperature][index],'go-')
    ax.plot(cnumber_range,lr_data[temperature][index],'ro-',alpha=0.1)
    ax.plot(cnumber_range,l1_data[temperature][index],'ro-',alpha=0.3)
    ax.plot(cnumber_range,l2_data[temperature][index],'ro-',alpha=0.5)
    ax.plot(cnumber_range,l3_data[temperature][index],'ro-')
    ax.set_yscale("log")
    ax.set_ylim(1e-12, 1)
    ax.legend(['Vapor','Distillate','Reflux','L1','L2','Bottom'],fontsize=18)
    ax.set_title('T, Reflux {:.2f} kmol/s'.format(cd_data_master[temperature]['Re'][index]),fontsize=18)

    ax.set_ylabel('Molar Flow (kmol/s)', color='K',fontsize=18)
    ax.set_xlabel('Carbon Number', color='K',fontsize=18)
    # ax.tick_params('y', colors='k',labelsize=18)
    # ax.tick_params('x', colors='k',labelsize=18)

    ax2.plot(cnumber_range,cd_x_data[temperature][index],'go-')
    ax2.plot(cnumber_range,rf_x_data[temperature][1][index],'co-')
    ax2.plot(cnumber_range,rf_x_data[temperature][2][index],'bo-')
    ax2.plot(cnumber_range,rf_x_data[temperature][3][index],'ro-')

    ax2.set_ylim(0, 0.2)
    ax2.legend(['Condenser','Stage 1','Stage 2','Stage 3'],fontsize=18)
    ax2.set_title('Liquid Composition (Mole)',fontsize=18)

    ax2.set_ylabel('Molar Fraction', color='K',fontsize=18)
    ax2.set_xlabel('Carbon Number', color='K',fontsize=18)

    ax.grid()
    ax2.grid()
    plt.show()

In [ ]:
widgets.interact(plot_distribution, index = widgets.IntSlider(
    value=0,
    min=0,
    max=20,
    step=1,
    description='Reflux:'),temperature=widgets.SelectionSlider(
    options=Trange,
    value=Trange[0],
    description='Temperature',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True)
);

In [ ]:
plot_distribution(20,523.15)